In [ ]:
# Data Cleaning Median Home Prices Files

#mhp original (added regions in excel) -> mhp regions (grouped by regions) -> mhp tiered (removed certain years to get the other data sets)

# I took the original data set from zillow, and using VLOOKUP, I divided each city / state into one of four predefined regions (Northeast, West, South, and Midwest)
# Using that dataset, I began my data cleanup:
import os
import csv
import pandas as pd

path=os.getcwd()
csv_path1=path+"\\Resources\Alex Files\median_home_price_zillow_region.csv"
housing_index = pd.read_csv(csv_path1, index_col="SizeRank")
housing_index.sort_values("SizeRank")

housing_index = housing_index.rename(columns={
    "Unnamed: 5": "Date",
})

region_index = housing_index.groupby(["Date"]).mean()

region_index = region_index.drop(columns=["RegionID"])

new = region_index.transpose()

# new.to_csv(r"..\Resources\Alex Files\median_home_price_zillow_region_tiered.csv")

narrowed_2015_2021 = new.iloc[228:304]

# narrowed_2015_2021.to_csv(r"..\Resources\Alex Files\median_home_price_zillow_region_tiered_narrowed_2015-2021.csv")

narrowed_2015_2019 = narrowed_2015_2021.iloc[0:62]
narrowed_2020 = narrowed_2015_2021.iloc[62:76]

# narrowed_2015_2019.to_csv(r"..\Resources\Alex Files\median_home_price_zillow_region_tiered_narrowed_2015-2019.csv")
# narrowed_2020.to_csv(r"..\Resources\Alex Files\median_home_price_zillow_region_tiered_narrowed_2020.csv")


# data sets that I am using: rental_analysis_zillow ; median_home_price_zillow_region_tiered_narrowed_2015-2019 ; median_home_price_zillow_region_tiered_narrowed_2020

In [ ]:
# Data Cleaning Rental File
import pandas as pd
import numpy as np
from datetime import datetime as dt
from pathlib import Path
import os
import sys
import csv
import datetime as dt
path=os.getcwd()

csv_path1=path+"..\Resources\Alex Files\rental_analysis_zillow.csv"
rental_file = pd.read_csv(csv_path1, index_col= "RegionName")

rental_file = rental_file.drop(columns=["SizeRank","RegionID"])

rent = rental_file.head(10)

rental = rent.transpose()

pct_change_rent = (rental.pct_change())
                   
rented = (pct_change_rent+1).cumprod()

rented.plot(figsize=(20,10), title = "Cumulative Returns on Rental Price: Metro Areas US")

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import scipy.stats as st
from scipy.stats import norm
from pathlib import Path
import os
import sys
import csv
import matplotlib.pyplot as plt
import math
import seaborn as sns
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
from dotenv import load_dotenv

%matplotlib inline

path=os.getcwd()
csv_path1=path+"\\Resources\Alex Files\median_home_price_zillow_region_tiered.csv"
house_prices = pd.read_csv(csv_path1, index_col="Date")

csv_path2=path+"\\Resources\Alex Files\median_home_price_zillow_region_tiered_narrowed_2015-2019.csv"
pre_2020 = pd.read_csv(csv_path2, index_col="Date")

csv_path3=path+"\\Resources\Alex Files\median_home_price_zillow_region_tiered_narrowed_2020.csv"
house_2020 = pd.read_csv(csv_path3, index_col="Date")

midwest_actual = house_2020.drop(columns = ["Northeast", "West", "South"])
west_actual = house_2020.drop(columns = ["Northeast", "Midwest", "South"])
south_actual = house_2020.drop(columns = ["Northeast", "Midwest", "West"])
northeast_actual = house_2020.drop(columns = ["South", "Midwest", "West"])

def price_prediction(self):
    region = input("Which Region would you like to see a prediction for?")
    if region == "Northeast":
        s = "West", "South", "Midwest"
    elif region == "West":
        s = "Northeast", "South", "Midwest"
    elif region == "South":
        s = "Northeast", "West", "Midwest"
    else: s = "Northeast", "West", "South"
    if region == "Northeast":
        d = 'Northeast'
    elif region == "West":
        d = "West"
    elif region == "South":
        d = "South"
    else: d = 'Midwest'
    h = str(d)
    regions = list(s)
    house_price_2020 = house_2020.drop(columns=regions)
    actual_YOY_price = house_price_2020.iloc[-1][d]
    sim=self.drop(columns=regions)
    log_returns = np.log(1 + sim.pct_change())
    u = log_returns.mean()
    var = log_returns.var()
    drift = u - (0.5 * var)
    stdev = log_returns.std()
    np.array(drift)
    np.array(stdev)
    drift.values
    stdev.values
    norm.ppf(0.95)
    x = np.random.rand(10, 2)
    norm.ppf(x)
    Z = norm.ppf(np.random.rand(10,2))
    t_intervals = 15
    iterations = 100
    monthly_returns = np.exp(drift.values + stdev.values * norm.ppf(np.random.rand(t_intervals, iterations)))
    S0 = sim.iloc[-1]
    price_list = np.zeros_like(monthly_returns)
    price_list[0] = S0
    for t in range(1, t_intervals):
        price_list[t] = price_list[t - 1] * monthly_returns[t]
    home = pd.DataFrame(price_list)
    pct_change = home.pct_change()
    prod = (pct_change + 1).cumprod()
    interval =prod.iloc[-1]
    mean = interval.mean()
    a= st.t.interval(0.95, len(interval)-1, loc=np.mean(interval), scale=st.sem(interval))
    last = sim.iloc[-1][d]
    lower_95 = a[0]
    upper_95 = a[1]
    lower = last*lower_95
    higher = last*upper_95
    average = mean * last
#   plt.figure(figsize=(10,6))
#   plt.plot(home)
#   plt.xlabel('Months Post Apr-2021')
#   plt.ylabel('Median Home Price')
#   plt.title(f"Median {h} Home Price Predicted 1 Year Out")
    plt.figure(figsize=(10,6))
    plt.plot(house_price_2020)
    plt.xlabel('Months Post Feb-2020') 
    plt.ylabel('Median Home Price')
    plt.title(f"Actual {h} Median Home Price 2020")
    plt.xticks(rotation=45)
    print(f"Your home in the {h} will be worth between ${lower:0.2f} and ${higher:0.2f} in 1 year's time with 95% certainty on current trends, barring any external factors ")
    return average, actual_YOY_price


def plot_setup(self):
    region = input("Which Region would you like to see a prediction for?")
    if region == "Northeast":
        s = "West", "South", "Midwest"
    elif region == "West":
        s = "Northeast", "South", "Midwest"
    elif region == "South":
        s = "Northeast", "West", "Midwest"
    else: s = "Northeast", "West", "South"
    if region == "Northeast":
        d = 'Northeast'
    elif region == "West":
        d = "West"
    elif region == "South":
        d = "South"
    else: d = 'Midwest'
    h = str(d)
    regions = list(s)
    house_price_2020 = house_2020.drop(columns=regions)
    actual_YOY_price = house_price_2020.iloc[-1][d]
    sim=self.drop(columns=regions)
    log_returns = np.log(1 + sim.pct_change())
    u = log_returns.mean()
    var = log_returns.var()
    drift = u - (0.5 * var)
    stdev = log_returns.std()
    np.array(drift)
    np.array(stdev)
    drift.values
    stdev.values
    norm.ppf(0.95)
    x = np.random.rand(10, 2)
    norm.ppf(x)
    Z = norm.ppf(np.random.rand(10,2))
    t_intervals = 15
    iterations = 100
    monthly_returns = np.exp(drift.values + stdev.values * norm.ppf(np.random.rand(t_intervals, iterations)))
    S0 = sim.iloc[-1]
    price_list = np.zeros_like(monthly_returns)
    price_list[0] = S0
    for t in range(1, t_intervals):
        price_list[t] = price_list[t - 1] * monthly_returns[t]
    home = pd.DataFrame(price_list)
    pct_change = home.pct_change()
    prod = (pct_change + 1).cumprod()
    interval =prod.iloc[-1]
    mean = interval.mean()
    a= st.t.interval(0.95, len(interval)-1, loc=np.mean(interval), scale=st.sem(interval))
    last = sim.iloc[-1][d]
    lower_95 = a[0]
    upper_95 = a[1]
    lower = last*lower_95
    higher = last*upper_95
    average = mean * last
    plt.figure(figsize=(10,6))
    plt.plot(home)
    plt.xlabel('Months Post Feb-2020')
    plt.ylabel('Median Home Price')
    plt.title(f"Median {h} Home Price Predicted 1 Year Out")
    return(home)

# Defining Plots for Dashboard
def midwestactual():
    """Housing Units Per Year."""
    midwest_actual
    return midwest_actual.hvplot.line(x="Date", y="", xlabel= "Month", ylabel="Home Price", ylim = (142000, 170000), title = "Midwest Region Median Home Prices").opts(xrotation=90)

def westactual():
    """Housing Units Per Year."""
    west_actual
    return west_actual.hvplot.line(x="Date", y="", xlabel= "Month", ylabel="Home Price", ylim = (330000, 385000), title = "West Region Median Home Prices").opts(xrotation=90)

def southactual():
    """Housing Units Per Year."""
    south_actual
    return south_actual.hvplot.line(x="Date", y="", xlabel= "Month", ylabel="Home Price", ylim = (142000, 170000), title = "South Region Median Home Prices").opts(xrotation=90)

def northeastactual():
    """Housing Units Per Year."""
    northeast_actual
    return northeast_actual.hvplot.line(x="Date", y="", xlabel= "Month", ylabel="Home Price", ylim = (200000, 230000), title = "Northeast Region Median Home Prices").opts(xrotation=90)

def midwest_plot():
    midwest
    return midwest.hvplot.line(x="", y="", xlabel= "Months Post Feb-2020", ylabel="Median Midwest Home Price", legend=False, title = "Median Midwest Home Price Predicted 1 Year Out").opts(framewise=True)

def south_plot():
    south
    return south.hvplot.line(x="", y="", xlabel= "Months Post Feb-2020", ylabel="Median South Home Price", legend=False, ylim = (142000, 160000), title = "Median South Home Price Predicted 1 Year Out")

def west_plot():
    west
    return west.hvplot.line(x="", y="", xlabel= "Months Post Feb-2020", ylabel="Median West Home Price", legend=False, ylim = (330000, 365000), title = "Median West Home Price Predicted 1 Year Out")

def northeast_plot():
    northeast
    return northeast.hvplot.line(x="", y="", xlabel= "Months Post Feb-2020", ylabel="Median Northeast Home Price", legend=False, ylim = (200000, 215000), title = "Median Northeast Home Price Predicted 1 Year Out")

In [ ]:
#Midwest
output_value_pre_2020_Midwest = price_prediction(pre_2020)

In [ ]:
midwest = plot_setup(pre_2020)

In [ ]:
#West
output_value_pre_2020_West = price_prediction(pre_2020)

In [ ]:
west = plot_setup(pre_2020)

In [ ]:
#Northeast
output_value_pre_2020_Northeast = price_prediction(pre_2020)

In [ ]:
northeast = plot_setup(pre_2020)

In [ ]:
#South
output_value_pre_2020_South = price_prediction(pre_2020)

In [ ]:
south = plot_setup(pre_2020)

In [ ]:
# Create a Title for the Dashboard
title = '##Median Home Price by Region for the US, 2015-2020, with Predicted and Actual Data Visualized'

prediction_avg_MW = output_value_pre_2020_Midwest[0]
actual_price_MW = output_value_pre_2020_Midwest[1]

actual_price_W = output_value_pre_2020_West[1]
prediction_avg_W = output_value_pre_2020_West[0]

actual_price_S = output_value_pre_2020_South[1]
prediction_avg_S = output_value_pre_2020_South[0]
actual_price_NE = output_value_pre_2020_Northeast[1]
prediction_avg_NE = output_value_pre_2020_Northeast[0]

# Create a tab layout for the dashboard
welcome_tab = pn.Column((title))
                        

midwest_comp = pn.Column(midwestactual(),
                         midwest_plot())

south_comp = pn.Column(southactual(),
                         south_plot())

northeast_comp = pn.Column(northeastactual(),
                         northeast_plot())

west_comp = pn.Column(westactual(),
                         west_plot())

# Create the dashboard
Home_Price_Dashboard = pn.Tabs(("Welcome", welcome_tab), 
("Midwest", midwest_comp),
("South", south_comp),
("West", west_comp),
("Northeast", northeast_comp))

In [ ]:
Midwest_diff = print(f"Difference against actual prices and predicted prices pre 2020 for the Midwest is {(actual_price_MW/prediction_avg_MW - 1)*100:0.2f}%")
West_diff = print(f"Difference against actual prices and predicted prices pre 2020 for the West is {(actual_price_W/prediction_avg_W - 1)*100:0.2f}%")
South_diff = print(f"Difference against actual prices and predicted prices pre 2020 for the South is {(actual_price_S/prediction_avg_S - 1)*100:0.2f}%")
Northeast_diff = print(f"Difference against actual prices and predicted prices pre 2020 for the Northeast is {(actual_price_NE/prediction_avg_NE - 1)*100:0.2f}%")
Home_Price_Dashboard